**In this notebook we have taken the tone code to train the bert model to identify the tone in the frame**

In [1]:
pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [2]:
import pandas as pd
from google.colab import files
upload = files.upload()


Saving df_tone_consensus.csv to df_tone_consensus.csv


In [3]:
!pip install transformers torch


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np


In [6]:
df = pd.read_csv('df_tone_consensus.csv')
df.head()

,Title,Tone Code Mode
0,"""BLACK HAWK DOWN""; THE MOVIE; SOMALIS IN TWIN ...",17
1,"""THE IMMIGRANT"" BLENDS STORY, DIALOGUE, CHARAC...",17
2,"""The economic gears of this country are depend...",17
3,"""We need new laws; we have to speak out; we ha...",17
4,"$50,000 per entry",19


In [8]:
df = df.dropna(subset=['Tone Code Mode'])

In [9]:
df = df.dropna(subset=['Title'])

In [10]:
len(df['Tone Code Mode'].unique())

3

In [12]:
df['Tone Code Mode'].unique()

array([17, 19, 18])

In [13]:
import pandas as pd
# Convert column values to integers
#df['first_tone_annotation'] = df['first_tone_annotation'].astype(int)

headlines = df['Title'].tolist()



In [14]:
mapping = {17: 0, 19: 1, 18: 2}
df['first_tone_annotation'] = df['Tone Code Mode'].map(mapping)

In [15]:
frames = df['first_tone_annotation'].tolist()

In [17]:
class NewsDataset(Dataset):
    def __init__(self, headlines, frames, tokenizer, max_length):
        self.headlines = headlines
        self.frames = frames
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.headlines)

    def __getitem__(self, idx):
        text = self.headlines[idx]
        label = self.frames[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, labels):
        ce_loss = nn.CrossEntropyLoss()(logits, labels)
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha * (1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss


In [20]:
# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(headlines, frames, test_size=0.2, random_state=42)

train_dataset = NewsDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = NewsDataset(val_texts, val_labels, tokenizer, max_length=128)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)


In [56]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    weight_decay=0.01,

)

# Custom model with Focal Loss
class CustomBERTModel(nn.Module):
    def __init__(self, model, num_labels, focal_loss=None):
        super(CustomBERTModel, self).__init__()
        self.model = model
        self.num_labels = num_labels
        self.focal_loss = focal_loss  # Focal Loss is optional

    def forward(self, input_ids, attention_mask, labels=None):
        # Forward pass through BERT
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        if labels is not None and self.focal_loss is not None:
            loss = self.focal_loss(logits, labels)
            return SequenceClassifierOutput(loss=loss, logits=logits)
        return logits



# Train model
#trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [31]:
trainer.save_model('/content/drive/MyDrive/your_directory/saved_model')
print("Model saved successfully to Google Drive.")


Model saved successfully to Google Drive.


In [29]:
trainer.save_model('D:/Admits/Usf Coursework all materials/Fall_Semester_3/Text Analytics/Projects/saved_model')

In [22]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.2973988354206085, 'eval_runtime': 95.7055, 'eval_samples_per_second': 9.028, 'eval_steps_per_second': 2.257, 'epoch': 10.0}


In [23]:
from sklearn.metrics import accuracy_score
import numpy as np

# Make predictions on the validation dataset
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)  # Get the predicted class labels
labels = predictions.label_ids                    # True labels

# Calculate micro accuracy
micro_accuracy = accuracy_score(labels, preds)
print(f"Micro Accuracy: {micro_accuracy:.4f}")


Micro Accuracy: 0.5961
